In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
normal_speakers = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

# read torgo test speaker

In [3]:
# Read the dataset
data_df = pd.read_csv('torgo.csv')
dataset_csv = load_dataset('csv', data_files='torgo.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [33]:
dataset_csv

DatasetDict({
    train: Dataset({
        features: ['session', 'audio', 'text', 'speaker_id'],
        num_rows: 16394
    })
})

In [37]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    return cleaned_text

# Apply the clean_text function to the 'text' column in data_df
data_df['text'] = data_df['text'].apply(clean_text)

In [38]:
import re

# Iterate through each text entry in the 'text' column of dataset_csv['train']
for i, text in enumerate(dataset_csv['train']['text']):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    
    # Update the 'text' column with the cleaned text
    dataset_csv['train']['text'][i] = cleaned_text

KeyboardInterrupt: 

In [39]:
data_df

,session,audio,text,speaker_id
0,F01-Session1-arrayMic-0006,/work/van-speech-nlp/data/torgo//F01/Session1/...,stick,F01
1,F01-Session1-arrayMic-0008,/work/van-speech-nlp/data/torgo//F01/Session1/...,except in the winter when the ooze or snow or ...,F01
2,F01-Session1-arrayMic-0009,/work/van-speech-nlp/data/torgo//F01/Session1/...,pat,F01
3,F01-Session1-arrayMic-0010,/work/van-speech-nlp/data/torgo//F01/Session1/...,up,F01
4,F01-Session1-arrayMic-0011,/work/van-speech-nlp/data/torgo//F01/Session1/...,meat,F01
...,...,...,...,...
16389,MC04-Session2-arrayMic-1022,/work/van-speech-nlp/data/torgo//MC04/Session2...,wart,MC04
16390,MC04-Session2-arrayMic-1023,/work/van-speech-nlp/data/torgo//MC04/Session2...,the family requests that flowers be omitted,MC04
16391,MC04-Session2-arrayMic-1024,/work/van-speech-nlp/data/torgo//MC04/Session2...,ship,MC04
16392,MC04-Session2-arrayMic-1026,/work/van-speech-nlp/data/torgo//MC04/Session2...,why,MC04


In [48]:
from collections import defaultdict

speaker_texts = defaultdict(list)

for i in range(len(data_df)):
    speaker_id = data_df.loc[i, 'speaker_id']
    # print(speaker_id)
    if speaker_id not in normal_speakers:
        text = data_df.loc[i, 'text']
        speaker_texts[speaker_id].append(text)


for speaker_id, texts in speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:1]:
        print(text)
    print()

F01: 228
stick

F03: 1075
beta

F04: 667
fee

M01: 739
when he speaks his voice is just a bit cracked and quivers a trifle

M02: 766
grow

M03: 800
now

M04: 652
trouble

M05: 573
swore



In [49]:
from collections import defaultdict

# Initialize a defaultdict to store the texts for each speaker
speaker_texts_train = defaultdict(list)

# Iterate through each test speaker
for test_speaker in atypical_speakers:
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    
    # Filter the DataFrame to get the data for the train split for the selected train speakers
    train_data = data_df[data_df['speaker_id'].isin(train_speaker)]
    
    # Append the texts from the filtered DataFrame to the corresponding speaker in the speaker_texts_train dictionary
    for text in train_data['text']:
        speaker_texts_train[test_speaker].append(text)

# Print the number of texts collected for each speaker
for speaker_id, texts in speaker_texts_train.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:1]:
        print(text)
    print()

F01: 15091
fee

F03: 14652
stick

F04: 14652
stick

M01: 14580
stick

M02: 14553
stick

M03: 14519
stick

M04: 14667
stick

M05: 14746
stick



# calculate unique words

In [57]:
unique_words_per_speaker_train = {}

for speaker_id, texts in speaker_texts_train.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker_train[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker_train.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 1561 unique words
F03: 1557 unique words
F04: 1557 unique words
M01: 1559 unique words
M02: 1557 unique words
M03: 1561 unique words
M04: 1559 unique words
M05: 1557 unique words


In [62]:
unique_words_per_speaker_train

{'F01': {'position',
  'wack',
  'symbols',
  'own',
  'valentines',
  'fall',
  'bull',
  'disappointing',
  'plant',
  'around',
  'wise',
  'muck',
  'bankers',
  'seam',
  'sit',
  'kept',
  'rattle',
  'meeting',
  'nine',
  'house',
  'crawl',
  'worse',
  'worry',
  'rust',
  'fathers',
  'not',
  'being',
  'play',
  'lost',
  'wonderful',
  'more',
  'immediately',
  'household',
  'geese',
  'stuff',
  'naval',
  'bust',
  'wanted',
  'swarm',
  'combed',
  'steel',
  'education',
  'way',
  'then',
  'zest',
  'snake',
  'wart',
  'horn',
  'yellow',
  'future',
  'spark',
  'asked',
  'minus',
  'winter',
  'lovely',
  'sergeant',
  'annoying',
  'confirmation',
  'cakes',
  'show',
  'old',
  'fast',
  'performance',
  'ramp',
  'wet',
  'am',
  'baseball',
  'contained',
  'unspecified',
  'serve',
  'young',
  'bran',
  'most',
  'ever',
  'glitter',
  'noted',
  'dressed',
  'ah',
  'calico',
  'at',
  'pink',
  'zero',
  'page',
  'issue',
  'tasks',
  'need',
  'any',

In [60]:
unique_words_per_speaker = {}

for speaker_id, texts in speaker_texts.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 190 unique words
F03: 768 unique words
F04: 669 unique words
M01: 605 unique words
M02: 622 unique words
M03: 645 unique words
M04: 450 unique words
M05: 718 unique words


In [61]:
unique_words_per_speaker

{'F01': {'a',
  'about',
  'air',
  'all',
  'alpha',
  'always',
  'an',
  'ancient',
  'and',
  'answers',
  'as',
  'banana',
  'bat',
  'be',
  'beard',
  'beat',
  'beta',
  'bit',
  'black',
  'born',
  'brown',
  'bubble',
  'bug',
  'but',
  'buttons',
  'car',
  'chair',
  'chin',
  'clings',
  'coat',
  'corn',
  'cracked',
  'dagger',
  'dark',
  'day',
  'dog',
  'double',
  'down',
  'dresses',
  'dug',
  'each',
  'ever',
  'except',
  'fair',
  'farm',
  'fee',
  'feed',
  'feeling',
  'feet',
  'floor',
  'flowing',
  'for',
  'form',
  'fox',
  'frock',
  'gadget',
  'giving',
  'glow',
  'go',
  'goat',
  'grandfather',
  'greasy',
  'grow',
  'had',
  'hair',
  'have',
  'he',
  'him',
  'himself',
  'his',
  'horn',
  'ice',
  'in',
  'is',
  'jacket',
  'jagged',
  'jumps',
  'just',
  'knee',
  'knew',
  'know',
  'language',
  'lazy',
  'left',
  'less',
  'likes',
  'long',
  'meat',
  'minus',
  'modern',
  'more',
  'my',
  'nearly',
  'ninety',
  'no',
  'obs

# vocabulary similarity

In [64]:
out_of_domain_words_per_speaker = {}

for speaker_id, unique_words in unique_words_per_speaker.items():
    overlap = len(unique_words & unique_words_per_speaker_train[speaker_id])
    similarity = overlap / len(unique_words) if len(unique_words) > 0 else 0
    out_of_domain_similarity = (1 - similarity)*100

    out_of_domain_words = {word for word in unique_words if word not in unique_words_per_speaker_train[speaker_id]}
    out_of_domain_words_per_speaker[speaker_id] = out_of_domain_words

    print(f"{speaker_id}: {out_of_domain_similarity:.2f}% dissimilarity with word_set_europarl")
    print(f"{speaker_id}: out_of_domain_words count: {len(out_of_domain_words)}")
    print(f"Out of domain words for {speaker_id}: {out_of_domain_words}")
    print()

F01: 0.53% dissimilarity with word_set_europarl
F01: out_of_domain_words count: 1
Out of domain words for F01: {'ninety'}

F03: 0.91% dissimilarity with word_set_europarl
F03: out_of_domain_words count: 7
Out of domain words for F03: {'prospects', 'sparkle', 'bird', 'wooden', 'wool', 'ozone', 'cupping'}

F04: 0.75% dissimilarity with word_set_europarl
F04: out_of_domain_words count: 5
Out of domain words for F04: {'lilyrare', 'bran', 'sweet', 'bite', 'dressed'}

M01: 0.50% dissimilarity with word_set_europarl
M01: out_of_domain_words count: 3
Out of domain words for M01: {'lead', 'cope', 'jaw'}

M02: 0.80% dissimilarity with word_set_europarl
M02: out_of_domain_words count: 5
Out of domain words for M02: {'drop', 'relax', 'shout', 'ignorance', 'oh'}

M03: 0.16% dissimilarity with word_set_europarl
M03: out_of_domain_words count: 1
Out of domain words for M03: {'liar'}

M04: 0.67% dissimilarity with word_set_europarl
M04: out_of_domain_words count: 3
Out of domain words for M04: {'min',

# clean the text

In [65]:
import re

cleaned_speaker_texts = {}

for speaker_id, texts in speaker_texts.items():
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
    cleaned_speaker_texts[speaker_id] = cleaned_texts

for speaker_id, texts in cleaned_speaker_texts.items():
    print(f"Speaker {speaker_id}:")
    for text in texts[:2]:  
        print(text)
    print()

Speaker F01:
stick
except in the winter when the ooze or snow or ice prevents

Speaker F03:
beta
stubble

Speaker F04:
fee
yet he still thinks as swiftly as ever

Speaker M01:
when he speaks his voice is just a bit cracked and quivers a trifle
trait

Speaker M02:
grow
feed

Speaker M03:
now
know

Speaker M04:
trouble
spark

Speaker M05:
swore
train



# Out of domain analysis

In [66]:
pattern = "keep_all"

In [67]:
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

for speaker in atypical_speakers:
    arpa_file = f"torgo_lm_arpa_files/{speaker}_3gram_{pattern}.arpa"
    print(arpa_file)
    model=kenlm.Model(arpa_file) 
    

torgo_lm_arpa_files/F01_3gram_keep_all.arpa
torgo_lm_arpa_files/F03_3gram_keep_all.arpa
torgo_lm_arpa_files/F04_3gram_keep_all.arpa
torgo_lm_arpa_files/M01_3gram_keep_all.arpa
torgo_lm_arpa_files/M02_3gram_keep_all.arpa
torgo_lm_arpa_files/M03_3gram_keep_all.arpa
torgo_lm_arpa_files/M04_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa


In [69]:
import kenlm
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    arpa_file = f"torgo_lm_arpa_files/{speaker}_3gram_{pattern}.arpa"
    print(arpa_file)
    model=kenlm.Model(arpa_file) 
    
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
torgo_lm_arpa_files/M05_3gram_keep_all.arpa
Speaker F01: Average Perplexity: 17.028228872205332
Speaker F03: Average Perplexity: 24.043141069921028
Speaker F04: Average Perplexity: 18.58511757127206
Speaker M01: Average Perplexity: 18.142184382541494
Speaker M02: Average Perplexity: 18.794837074219913
Speaker M03: Average Perplexity: 18.568670409421134
Speaker M04: Average Perplexity: 17.49000922837935
Speaker M05: Average Perplexity: 21.27719734033508


Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/torgo_lm_arpa_files/M05_3gram_keep_all.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85-

In [71]:
mean_perplexity = sum(perplexities_per_speaker.values()) / len(perplexities_per_speaker)

print("Mean Perplexity:", mean_perplexity)

Mean Perplexity: 19.241173243536924


# output unigram

libri.apra
